In [ ]:
# Librerías estándar de Python
import datetime  # Manejo de fechas y horas
import pandas as pd  # Manipulación y análisis de datos en DataFrames
import numpy as np  # Operaciones numéricas y manejo eficiente de arrays
import sys  # Acceso a variables y funciones del sistema
import re  # Expresiones regulares para procesamiento de texto
import os  # Operaciones del sistema de archivos
import src  # Permite importar módulos del proyecto (si existe un __init__.py)

# Añadir la carpeta raíz del proyecto al sys.path para importar módulos personalizados
sys.path.append(os.path.abspath("c:/Users/osmarrincon/Documents/capresoca-data-automation"))
#sys.path.append(os.path.abspath("D:\Proyectos Python\capresoca-data-automation"))  # Ruta alternativa (comentada)

# Importar función y clase personalizada del proyecto
from src.file_loader import cargar_maestros_ADRES  # Función para cargar archivos maestros ADRES
from src.data_cleaning import DataCleaner      # Clase para limpiar y normalizar población

# Instanciar el limpiador de población para su uso posterior
cleaner = PoblacionCleaner()

ImportError: cannot import name 'PoblacionCleaner' from 'src.data_cleaning' (c:\Users\osmarrincon\Documents\capresoca-data-automation\src\data_cleaning.py)

# Rutas y varaibles

In [ ]:
R_Pila_Validada = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\07_Julio\15\Dataframe 15-07-2025.xlsx"
R_Maestro__EPSC25 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0014072025.TXT"
R_Maestro__EPS025 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-2\EPS025MS0014072025.TXT"

S_Excel = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\07_Julio\15\Prueba.xlsx"

# Cargue Dataframes

In [ ]:
# Cargar y combinar los maestros
maestro_ADRES = cargar_maestros_ADRES(R_Maestro__EPS025, R_Maestro__EPSC25)
Pila_Validada = pd.read_excel(R_Pila_Validada, sheet_name="Sheet1", header=0, dtype=str)

In [ ]:
# Duplicar la columna "MARCASISBENIV+MARCASISBENIII_2" y nombrarla "MARCASISBENIV+MARCASISBENIII"
maestro_ADRES["MARCASISBENIV+MARCASISBENIII_2"] = \
    maestro_ADRES["MARCASISBENIV+MARCASISBENIII"]

In [ ]:
jerarquia = ['9', '17', '28', '2', '1']

def limpiar_poblacion(valor):
    if pd.isna(valor):
        return valor

    original = valor  # Para posible trazabilidad

    # 1. Reemplazar si es exactamente SIV(Dnn)
    if re.fullmatch(r"SIV\(D\d{2}\)", valor.strip()):
        return "Sisben D"

    # 2. Eliminar sufijos -SIV(...) y -SIII(...)
    valor = re.sub(r"-SIV\([^\)]*\)", "", valor)
    valor = re.sub(r"-SIII\([^\)]*\)", "", valor)

    # 3. Si es solo SIII(...), eliminar todo
    if re.fullmatch(r"SIII\([^\)]*\)", valor.strip()):
        return ""

    # 4. Eliminar prefijo "LC(0)-"
    if valor.startswith("LC(0)-"):
        valor = valor.replace("LC(0)-", "")

    # 5. Si queda solo "LC(0)", eliminar
    if valor.strip() == "LC(0)":
        return ""

    # 6. Normalizar LC(...) con jerarquía
    def jerarquia_lc(match):
        contenido = match.group(1)
        valores = [v.strip() for v in contenido.split('|')]

        if len(valores) == 1:
            return f"LC({valores[0]})"

        for prioridad in jerarquia:
            if prioridad in valores:
                return f"LC({prioridad})"

        return ""  # Eliminar si no hay valor válido

    valor = re.sub(r"LC\(([^\)]*)\)", jerarquia_lc, valor)

    return valor.strip()

# Aplicar la función
maestro_ADRES["MARCASISBENIV+MARCASISBENIII"] = maestro_ADRES["MARCASISBENIV+MARCASISBENIII"].apply(limpiar_poblacion)

# contrucción S1 movilidad

## Pila

In [ ]:
# Definir las columnas del nuevo DataFrame
new_columns = [
    "ENT_ID", "TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "AFL_PRIMER_APELLIDO", "AFL_SEGUNDO_APELLIDO",
    "AFL_PRIMER_NOMBRE", "AFL_SEGUNDO_NOMBRE", "AFL_FECHA_NACIMIENTO", "TPS_GNR_ID", "TPS_IDN_ID_2",
    "HST_IDN_NUMERO_IDENTIFICACION_2", "AFL_PRIMER_APELLIDO_2", "AFL_SEGUNDO_APELLIDO_2", "AFL_PRIMER_NOMBRE_2",
    "AFL_SEGUNDO_NOMBRE_2", "AFL_FECHA_NACIMIENTO_2", "TPS_GNR_ID_2", "DPR_ID", "MNC_ID", "ZNS_ID",
    "FECHA_AFILIACION_MOVILIDAD", "TPS_GRP_PBL_ID", "TPS_NVL_SSB_ID", "TIPO_TRASLADO", "CND_AFL_SBS_METODOLOGIA",
    "CND_AFL_SBS_SUBGRUPO_SIV", "CON_DISCAPACIDAD", "TPS_IDN_CF_ID", "HST_IDN_NUMERO_CF_IDENTIFICACION",
    "TPS_PRN_ID", "TPS_AFL_ID", "TPS_ETN_ID", "NOM_RESGUARDO_INDIGENA",
    "PAIS_NACIMIENTO", "LUGAR_NACIMIENTO", "NACIONALIDAD", "SEXO_IDENTIFICACION", "TIPO_DISCAPACIDAD"
]

# Filtrar registros donde 'Movilidad' contiene 'S1'
mask = Pila_Validada["Movilidad"].str.contains("S1", na=False)
filtered = Pila_Validada.loc[mask]

# Crear el nuevo DataFrame S1 con valores vacíos
S1 = pd.DataFrame('', index=filtered.index, columns=new_columns)

# Asignar los valores requeridos
S1.loc[:, "TPS_IDN_ID"] = filtered["Tipo Documento Cotizante"]
S1.loc[:, "HST_IDN_NUMERO_IDENTIFICACION"] = filtered["N° Identificación Cotizante"]
S1.loc[:, "FECHA_AFILIACION_MOVILIDAD"] = filtered["Fecha envio"]
S1.loc[:, "CND_AFL_SBS_SUBGRUPO_SIV"] = filtered["Población_2"]

# Mostrar cantidad de registros, columnas y porcentaje de vacíos por columna
print(f"Registros: {S1.shape[0]}, Columnas: {S1.shape[1]}")
print("Porcentaje de vacíos por columna:")
print(S1.isin(['', None, np.nan]).mean() * 100)

In [ ]:
S1["Where"] = "Pila"

## Beneficairios

In [ ]:
import pandas as pd

# ----------------------------------------------------
# Debug: mostrar nombres de columnas antes de cualquier operación
# ----------------------------------------------------
print(">> S1 columns antes de strip():")
print([repr(c) for c in S1.columns.tolist()])
print(">> maestro_ADRES columns antes de strip():")
print([repr(c) for c in maestro_ADRES.columns.tolist()])

# ----------------------------------------------------
# Limpiar espacios invisibles en los nombres de columna
# ----------------------------------------------------
S1.columns            = S1.columns.str.strip()
maestro_ADRES.columns = maestro_ADRES.columns.str.strip()

# ----------------------------------------------------
# Debug: volver a mostrar nombres de columnas después de strip()
# ----------------------------------------------------
print(">> S1 columns después de strip():")
print([repr(c) for c in S1.columns.tolist()])
print(">> maestro_ADRES columns después de strip():")
print([repr(c) for c in maestro_ADRES.columns.tolist()])

# ----------------------------------------------------
# 0) Cantidad de registros y columnas en S1
# ----------------------------------------------------
print(f"Registros: {S1.shape[0]}, Columnas: {S1.shape[1]}")

# ----------------------------------------------------
# 1) Preparar sólo las claves de S1 (cabezas de familia),
#    incluyendo FECHA_AFILIACION_MOVILIDAD para luego propagarla
# ----------------------------------------------------
s1_keys = (
    S1[["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "FECHA_AFILIACION_MOVILIDAD"]]
    .drop_duplicates(subset=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"])
)

# ----------------------------------------------------
# 2) Emparejar S1 (cabezas) con maestro_ADRES (beneficiarios)
#    - Usamos sufijos para distinguir columnas duplicadas
# ----------------------------------------------------
beneficiarios = (
    maestro_ADRES
    .merge(
        s1_keys,
        left_on=["TPS_IDN_ID_CF", "HST_IDN_NUMERO_IDENTIFICACION_CF"],
        right_on=["TPS_IDN_ID",    "HST_IDN_NUMERO_IDENTIFICACION"],
        how="inner",
        suffixes=("_master", "_s1")
    )
)

# Debug: ver columnas que resultaron de este merge
print(">> beneficiarios columns:")
print(beneficiarios.columns.tolist())

# ----------------------------------------------------
# 3) Construir los nuevos registros con las columnas solicitadas,
#    propagando FECHA_AFILIACION_MOVILIDAD desde la familia cabeza
# ----------------------------------------------------
nuevos = pd.DataFrame({
    # ID del beneficiario (de maestro_ADRES con sufijo _master)
    "TPS_IDN_ID":                        beneficiarios["TPS_IDN_ID_master"],
    "HST_IDN_NUMERO_IDENTIFICACION":     beneficiarios["HST_IDN_NUMERO_IDENTIFICACION_master"],
    # Subgrupo según MARCASISBENIV+MARCASISBENIII
    "CND_AFL_SBS_SUBGRUPO_SIV":          beneficiarios["MARCASISBENIV+MARCASISBENIII"],
    # IDs de la cabeza de familia
    "TPS_IDN_CF_ID":                     beneficiarios["TPS_IDN_ID_CF"],
    "HST_IDN_NUMERO_CF_IDENTIFICACION":  beneficiarios["HST_IDN_NUMERO_IDENTIFICACION_CF"],
    # TPS_PRN_ID del beneficiario
    "TPS_PRN_ID":                        beneficiarios["TPS_PRN_ID"],
    # Marcamos estos nuevos registros con la etiqueta fija
    "Where":                             "Beneficiarios Pila",
    # Propagamos la misma fecha de movilidad que tenía la cabeza de familia
    "FECHA_AFILIACION_MOVILIDAD":        beneficiarios["FECHA_AFILIACION_MOVILIDAD"],
})

# ----------------------------------------------------
# 4) Añadir los nuevos registros al final de S1
# ----------------------------------------------------
S1 = pd.concat([S1, nuevos], ignore_index=True)

# ----------------------------------------------------
# 5) Mostrar resumen final
# ----------------------------------------------------
print(f"Registros después del concat: {S1.shape[0]}")
print(f"Columnas después del concat:  {S1.shape[1]}")


### Fechas de Beneficairios

## Sisben Maestro ADRES

In [ ]:
# Realizar el mapeo de los valores de maestro_ADRES["MARCASISBENIV+MARCASISBENIII"]
# a S1["CND_AFL_SBS_SUBGRUPO_SIV"] solo donde S1["Where"] == "Beneficiarios Pila"

# Crear un diccionario para búsqueda rápida por ID
maestro_map = maestro_ADRES.set_index(["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"])["MARCASISBENIV+MARCASISBENIII"]

# Actualizar S1 usando .apply para mantener el nombre de las columnas
def actualizar_subgrupo(row):
    if row.get("Where") == "Beneficiarios Pila":
        key = (row["TPS_IDN_ID"], row["HST_IDN_NUMERO_IDENTIFICACION"])
        valor = maestro_map.get(key, row["CND_AFL_SBS_SUBGRUPO_SIV"])
        return valor
    return row["CND_AFL_SBS_SUBGRUPO_SIV"]

S1["CND_AFL_SBS_SUBGRUPO_SIV"] = S1.apply(actualizar_subgrupo, axis=1)

## Datos del Maestro al S1

In [ ]:
import pandas as pd

# 1) Defino cómo se llaman en maestro → cómo quiero que queden en S1
column_mapping = {
    "AFL_PAIS_NACIMIENTO":     "PAIS_NACIMIENTO",
    "AFL_MUNICIPIO_NACIMIENTO":"LUGAR_NACIMIENTO",
    "AFL_NACIONALIDAD":        "NACIONALIDAD",
    "AFL_SEXO_IDENTIFICACION": "SEXO_IDENTIFICACION",
    "AFL_DISCAPACIDAD":        "TIPO_DISCAPACIDAD",
    "DPR_ID":                  "DPR_ID",
    "MNC_ID":                  "MNC_ID",
    "ZNS_ID":                  "ZNS_ID",
    "TPS_AFL_ID":              "TPS_AFL_ID",
    # (y las que ya tenías antes…)
    "AFL_PRIMER_APELLIDO":    "AFL_PRIMER_APELLIDO",
    "AFL_SEGUNDO_APELLIDO":   "AFL_SEGUNDO_APELLIDO",
    "AFL_PRIMER_NOMBRE":      "AFL_PRIMER_NOMBRE",
    "AFL_SEGUNDO_NOMBRE":     "AFL_SEGUNDO_NOMBRE",
    "AFL_FECHA_NACIMIENTO":   "AFL_FECHA_NACIMIENTO",
    "TPS_GNR_ID":             "TPS_GNR_ID",
}

join_keys = ["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"]

# 2) Preparo el maestro: me quedo solo con las columnas que necesito y
#    las renombro para que coincidan con S1:
maestro_subset = (
    maestro_ADRES[list(column_mapping.keys()) + join_keys]
    .rename(columns=column_mapping)
)

# 3) Hago el merge contra S1; pandas añadirá un sufijo solo a las columnas
#    del maestro (porque ya existen en S1):
S1_merged = S1.merge(
    maestro_subset,
    on=join_keys,
    how="left",
    suffixes=("", "_maestro")
)

# 4) Ahora, para cada columna de destino 'col', relleno S1[col] solo donde
#    esté vacío usando la versión '_maestro', y luego elimino esa columna:
for col in column_mapping.values():
    maestro_col = f"{col}_maestro"
    if maestro_col in S1_merged:
        S1_merged[col] = S1_merged[col].replace("", pd.NA) \
                                       .fillna(S1_merged[maestro_col]) \
                                       .fillna("")  # opcional volver a cadena vacía
        S1_merged.drop(columns=maestro_col, inplace=True)

# 5) S1_merged es tu S1 final, con el mismo esquema de columnas y
#    solo rellenando lo que estaba vacío:
S1 = S1_merged

In [ ]:
# Asignar los valores de las columnas base a las columnas "_2" en S1
cols_base = [
    "TPS_IDN_ID",
    "HST_IDN_NUMERO_IDENTIFICACION",
    "AFL_PRIMER_APELLIDO",
    "AFL_SEGUNDO_APELLIDO",
    "AFL_PRIMER_NOMBRE",
    "AFL_SEGUNDO_NOMBRE",
    "AFL_FECHA_NACIMIENTO",
    "TPS_GNR_ID"
]
cols_2 = [
    "TPS_IDN_ID_2",
    "HST_IDN_NUMERO_IDENTIFICACION_2",
    "AFL_PRIMER_APELLIDO_2",
    "AFL_SEGUNDO_APELLIDO_2",
    "AFL_PRIMER_NOMBRE_2",
    "AFL_SEGUNDO_NOMBRE_2",
    "AFL_FECHA_NACIMIENTO_2",
    "TPS_GNR_ID_2"
]

for base, col2 in zip(cols_base, cols_2):
    S1[col2] = S1[base]

In [ ]:
import re

def extract_nvl_ssb(subgrupo):
    if pd.isna(subgrupo):
        return ""
    lc_match = re.match(r"LC\((\d+)\)", subgrupo)
    if lc_match:
        return lc_match.group(1)
    siv_match = re.match(r"SIV\([A-Z0-9]+\)", subgrupo)
    if siv_match:
        return "5"
    return ""

S1["TPS_GRP_PBL_ID"] = S1["CND_AFL_SBS_SUBGRUPO_SIV"].apply(extract_nvl_ssb)

In [ ]:
# Asignación condicional según las reglas descritas
def assign_values(row):
    grp_pbl_id = row["TPS_GRP_PBL_ID"]
    subgrupo = row["CND_AFL_SBS_SUBGRUPO_SIV"]
    if grp_pbl_id != "5":
        row["TPS_NVL_SSB_ID"] = "N"
        row["CND_AFL_SBS_METODOLOGIA"] = "3"
        row["CND_AFL_SBS_SUBGRUPO_SIV"] = ""
    else:
        # Extraer la letra y número de SIV(xxx)
        match = re.match(r"SIV\(([A-Z])(\d{2})\)", subgrupo)
        if match:
            letra = match.group(1)
            numero = match.group(2)
            if letra in ["A", "B"]:
                row["TPS_NVL_SSB_ID"] = "1"
                row["CND_AFL_SBS_METODOLOGIA"] = "2"
            elif letra == "C":
                row["TPS_NVL_SSB_ID"] = "2"
                row["CND_AFL_SBS_METODOLOGIA"] = "2"
            row["CND_AFL_SBS_SUBGRUPO_SIV"] = f"{letra}{numero}"
        else:
            # Si no coincide con el patrón, dejar vacío
            row["TPS_NVL_SSB_ID"] = ""
            row["CND_AFL_SBS_METODOLOGIA"] = ""
            row["CND_AFL_SBS_SUBGRUPO_SIV"] = ""
    return row

S1 = S1.apply(assign_values, axis=1)

# Guardamos Dataframes

In [ ]:

with pd.ExcelWriter(S_Excel, engine="openpyxl") as writer:
    S1.to_excel(writer, sheet_name="S1", index=False)
    maestro_ADRES.to_excel(writer, sheet_name="maestro_ADRES", index=False)

print("Archivo Excel guardado en:", S_Excel)

In [ ]:
maestro_ADRES.columns